# `pandas` Part II - Group and aggregation

This document continues to cover data manipulation with `pandas`, including grouping, aggregating, and reorganizing data.

## Grouping data

Grouping together that are in the same category to aggregate over rows in each category.

Useful in 
- performing large operations, and
- summarizing trends in a dataset.

Say we have a dataset with baby naming frequency throughout the years.  Perhaps we are first interested in 

- how many babies are born in each year? (Good indicator of societal confidence..)

```{figure} ../img/pandas-group-schema.png
---
width: 80%
name: pandas-group
---
Example of aggregation in `pandas` {cite:p}`lau2023learning`
```

In [1]:
import pandas as pd

baby = pd.read_csv('../data/ssa-names.csv.zip')

In [ ]:
# number of total babies
baby['Count'].sum()

### Grouping and aggregating

**How many babies are born each year?**

In [ ]:
counts_by_year = baby.groupby('Year')['Count'].sum()

### A general recipe for grouping

```python
(baby                # the dataframe
 .groupby('Year')    # column(s) to group
 ['Count']           # column(s) to aggregate
 .sum()              # how to aggregate
)

# general form
dataframe.groupby(column_name).agg(aggregation_function)
```

### Grouping by multiple attributes

**How many female and male babies are born each year?**

In [ ]:
counts_by_year_and_sex = baby.groupby(['Year', 'Sex'])['Count'].sum()
counts_by_year_and_sex

### Aggregating by a custom function

**What about number of unique names by year?**

In [ ]:
def count_unique(names):
    return len(names.unique())

unique_names_by_year = (baby
 .groupby('Year')
 ['Name']
 .agg(count_unique) # aggregate using the custom count_unique function
)
unique_names_by_year

## Apply
The `Series.apply()` function applies an arbitrary function on each row entry.

**Retrieve first letter of name**

In [ ]:
def get_first_letter(s):
    return s[0]  # assumes string input

In [ ]:
names = baby['Name']
names.apply(get_first_letter)

**Number of letters in name**

### Quick word about `apply()` effectiveness

The `apply()` function is flexible, accommodating custom operations.  But it is *slow*.

In [ ]:
def does_nothing(year):
    return year / 10 * 10

In [61]:
years = baby['Year']

In [63]:
%timeit years / 10 * 10

85.8 ms ± 8.66 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [64]:
%timeit years.apply(does_nothing)

2.53 s ± 142 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Pivoting
Pivoting is one way to organize and present data, by arranging the results of a group and aggregation when grouping with two columns.

```{figure} ../img/pandas-pivot.png
---
width: 80%
name: pandas-pivot
---
Example of pivoting in `pandas` (Data 100)
```

In [ ]:
mf_pivot = pd.pivot_table(
    baby,
    index='Year',   # Column to turn into new index
    columns='Sex',  # Column to turn into new columns
    values='Count', # Column to aggregate for values
    aggfunc='sum')    # Aggregation function
mf_pivot

## Melting
Melting is the "reverse" of pivoting, transforming *wide* tables into *long* tables.

In [ ]:
mf_long = mf_pivot.reset_index().melt(
    id_vars='Year', # column that uniquely identifies a row (can be multiple)
    var_name='Sex', # name for the new column created by melting
    value_name='Count' # name for new column containing values from melted columns
)
mf_long

*Why do we need* `reset_index()`?

## Practice 3

Using the baby names data, find the names with most occurrences in each year for both sexes.

In [8]:
baby.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6311504 entries, 0 to 6311503
Data columns (total 5 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   State   object
 1   Sex     object
 2   Year    int64 
 3   Name    object
 4   Count   int64 
dtypes: int64(2), object(3)
memory usage: 240.8+ MB


In [29]:
# First attempt: most occurred names over all states for each year and sex
def max_count_name(column):
    max_index = column.idxmax()
    return baby.loc[max_index, 'Name']

baby.groupby(['Year', 'Sex'])['Count'].agg(max_count_name)

Year  Sex
1910  F        Mary
      M        John
1911  F        Mary
      M        John
1912  F        Mary
              ...  
2019  M        Noah
2020  F      Olivia
      M        Noah
2021  F      Olivia
      M        Noah
Name: Count, Length: 224, dtype: object

In [4]:
baby.loc[baby.groupby(['Year', 'Sex'])['Count'].idxmax()]

,State,Sex,Year,Name,Count
4917850,PA,F,1910,Mary,2913
5036503,PA,M,1910,John,1326
4918257,PA,F,1911,Mary,3188
5036756,PA,M,1911,John,1672
4918688,PA,F,1912,Mary,4106
...,...,...,...,...,...
6302995,CA,M,2019,Noah,2677
6139273,CA,F,2020,Olivia,2350
6305858,CA,M,2020,Noah,2625
6142887,CA,F,2021,Olivia,2395


In [30]:
# Second attempt: finding the total number of babies by year, sex, and name
# then group to find the most occurred name in each year for each sex
baby_allstates = baby.groupby(['Year', 'Sex', 'Name'])['Count'].sum().reset_index()
baby_allstates.loc[baby_allstates.groupby(['Year', 'Sex'])['Count'].idxmax()]

,Year,Sex,Name,Count
724,1910,F,Mary,22848
1444,1910,M,John,11450
2489,1911,F,Mary,24390
3245,1911,M,John,13446
4439,1912,F,Mary,32304
...,...,...,...,...
631732,2019,M,Liam,20578
637395,2020,F,Olivia,17641
641096,2020,M,Liam,19777
646827,2021,F,Olivia,17728


In [17]:
# This attempt does NOT work, because max() applies to the columns 'Count' and 'Name' independently.
baby_allstates.groupby(['Year', 'Sex'])[['Count', 'Name']].max()

Count    Name
Year Sex               
1910 F    22848    Zula
     M    11450  Zennie
1911 F    24390    Zula
     M    13446     Zeb
1912 F    32304    Zula
...         ...     ...
2019 M    20578    Zyon
2020 F    17641   Zyrah
     M    19777    Zyon
2021 F    17728    Zyra
     M    20272   Zyrus

[224 rows x 2 columns]

Using the meteorite data from the `Meteorite_Landings.csv` file, 

1. use `groupby` to examine the number of meteors recorded each year.
2. use `groupby` to find the heaviest meteorite from each year and report its name and mass.
3. create a pivot table that shows for each year
    - the number of meteorites, and
    - the 95th percentile of meteorite mass.
4. create a pivot table to compare for each year
    - the 5%, 25%, 50%, 75%, and 95% percentile of the mass column for the meteorites that were found versus observed falling.
5. melt the two tables above to create a *long*-format table.

In [18]:
meteor = pd.read_csv('../data/Meteorite_Landings.csv')

In [ ]:
meteor.info()

In [27]:
import numpy as np
pivot_table_3 = pd.pivot_table(meteor,
                               index='year',
                               values='mass (g)',
                               aggfunc=[len, lambda x: np.quantile(x, 0.95)])
pivot_table_3.columns = ['num. meteorites', '95th percentile mass']

In [28]:
pivot_table_3

,num. meteorites,95th percentile mass
year,,
860.0,1,472.00
920.0,1,NaN
1399.0,1,107000.00
1490.0,1,103.30
1491.0,1,127000.00
...,...,...
2010.0,1005,2071.60
2011.0,713,3381.80
2012.0,234,3639.45
